# Otimização Biobjetivo com Método ε-restrito usando VNS
Este notebook implementa o método epsilon-restrito para resolução de um problema biobjetivo de manutenção de equipamentos, utilizando a abordagem VNS (Variable Neighborhood Search).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import csv

class Struct:
    pass

class MyVNS:
    def shake(self, x, k, probdata):
        y = copy.deepcopy(x)
        ridx1 = np.random.randint(0, probdata.n)
        ridx2 = np.random.randint(0, probdata.n)
        blockSize = 50
        startIndex = np.random.randint(0, probdata.n - blockSize)
        blockIndices = range(startIndex, startIndex + blockSize)
        if k == 1:
            y.solution[ridx1] = np.random.choice([j for j in [0, 1, 2] if j != x.solution[ridx1]])
        elif k == 2:
            y.solution[ridx1], y.solution[ridx2] = x.solution[ridx2], x.solution[ridx1]
        elif k == 3:
            currentPlans = [x.solution[i] for i in blockIndices]
            currentPlan = max(set(currentPlans), key=currentPlans.count)
            newPlan = np.random.choice([j for j in [0, 1, 2] if j != currentPlan])
            for i in blockIndices:
                y.solution[i] = newPlan
        return y

    def neighborhoodChange(self, x, y, k):
        if y.fitness < x.fitness:
            x = copy.deepcopy(y)
            k = 1
        else:
            k += 1
        return x, k

    def firstImprovement(self, w1, w2, x, probdata, fobj, k):
        neighborhood_size = 100
        for j in range(neighborhood_size):
            current_solution = self.shake(x, k, probdata)
            if fobj(w1, w2, current_solution, probdata).fitness < fobj(w1, w2, x, probdata).fitness:
                return current_solution
        return x

class MyProblem:
    def fobj1(self, x, probdata):
        return np.sum(x.solution)

    def fobj2(self, x, probdata):
        fitness = 0
        for i in range(probdata.n):
            j = x.solution[i]
            fitness += probdata.dipij[i, j]
        return fitness

    def soma_ponderada(self, w1, w2, x, probdata):
        min_f1, max_f1 = 0, 1000
        min_f2, max_f2 = 1048.17, 1745.49
        f1 = self.fobj1(x, probdata)
        f2 = self.fobj2(x, probdata)
        x.f1 = f1
        x.f2 = f2
        x.f1norm = (f1 - min_f1) / (max_f1 - min_f1)
        x.f2norm = (f2 - min_f2) / (max_f2 - min_f2)
        x.fitness = w1 * x.f1norm + w2 * x.f2norm
        return x

    def sol_inicial(self, probdata, apply_constructive_heuristic):
        x = Struct()
        x.solution = np.random.randint(0, 3, size=(probdata.n))
        if apply_constructive_heuristic:
            for i in range(probdata.n):
                var = np.var(probdata.dipij[i, :])
                x.solution[i] = 2 if var > 0.5 else 0
        return x

    def probF(self, eta, beta, t):
        return 1 - np.exp(-(t / eta) ** beta)

    def probdef(self):
        n = 500
        equip_db = np.zeros((n, 4))
        with open('arquivos_tc/EquipDB.csv', newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for idx, row in enumerate(reader):
                equip_db[idx] = np.array([float(x) for x in ','.join(row).split(',')])

        mpdb = np.zeros((3, 3))
        c = np.zeros(3)
        with open('arquivos_tc/MPDB.csv', newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for idx, row in enumerate(reader):
                values = [float(x) for x in ','.join(row).split(',')]
                mpdb[idx] = np.array(values)
                c[idx] = values[-1]

        cluster = np.zeros((4, 3))
        with open('arquivos_tc/ClusterDB.csv', newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for idx, row in enumerate(reader):
                cluster[idx] = np.array([float(x) for x in ','.join(row).split(',')])

        dipij = np.zeros((n, 3))
        for i in range(n):
            clusterid = int(equip_db[i, 2] - 1)
            eta, beta = cluster[clusterid, 1], cluster[clusterid, 2]
            t0, di = equip_db[i, 1], equip_db[i, 3]
            for j in range(3):
                k = mpdb[j, 1]
                Ft0 = self.probF(eta, beta, t0)
                Ft0kt = self.probF(eta, beta, t0 + k * 5)
                dipij[i, j] = di * (Ft0kt - Ft0) / (1 - Ft0)

        probdata = Struct()
        probdata.equip_db = equip_db
        probdata.mpdb = mpdb
        probdata.c = c
        probdata.n = n
        probdata.dipij = dipij
        return probdata


In [2]:
class EpsilonRestrictedVNS:
    def __init__(self, myVNS, myProblem, probdata, max_avaliacoes=2000, kmax=3):
        self.myVNS = myVNS
        self.myProblem = myProblem
        self.probdata = probdata
        self.max_avaliacoes = max_avaliacoes
        self.kmax = kmax

    def busca_local_epsilon(self, x, epsilon):
        num_aval = 0
        while num_aval < self.max_avaliacoes:
            k = 1
            while k <= self.kmax:
                y = self.myVNS.shake(x, k, self.probdata)
                y = self.myVNS.firstImprovement(1, 0, x, self.probdata, self.myProblem.soma_ponderada, k)
                y = self.myProblem.soma_ponderada(1, 0, y, self.probdata)
                if y.f2 <= epsilon and y.f1 < x.f1:
                    x = copy.deepcopy(y)
                    k = 1
                else:
                    k += 1
                num_aval += 1
        return x

    def executar_execucoes(self, epsilons, n_execucoes=5, usar_heuristica=True):
        todas_fronteiras_f1 = []
        todas_fronteiras_f2 = []
        for execucao in range(n_execucoes):
            print(f"Execução {execucao + 1}")
            f1s, f2s = [], []
            for eps in epsilons:
                print(f"ε = {eps:.2f}")
                x = self.myProblem.sol_inicial(self.probdata, usar_heuristica)
                x = self.myProblem.soma_ponderada(1, 0, x, self.probdata)
                x = self.busca_local_epsilon(x, eps)
                f1s.append(x.f1)
                f2s.append(x.f2)
            todas_fronteiras_f1.append(f1s)
            todas_fronteiras_f2.append(f2s)
        return todas_fronteiras_f1, todas_fronteiras_f2


In [3]:
# Execução do algoritmo epsilon-restrito
myProblem = MyProblem()
myVNS = MyVNS()
probdata = myProblem.probdef()

epsilons = np.linspace(1048.17, 1745.49, 20)
epsilon_vns = EpsilonRestrictedVNS(myVNS, myProblem, probdata)
todas_f1, todas_f2 = epsilon_vns.executar_execucoes(epsilons, n_execucoes=5)


Execução 1
ε = 1048.17
ε = 1084.87


KeyboardInterrupt: 

In [ ]:
# Plotagem dos resultados
plt.figure(figsize=(10, 6))
for i in range(5):
    plt.plot(todas_f1[i], todas_f2[i], marker='o', label=f'Execução {i+1}')
plt.title('Fronteiras de Pareto - Método ε-restrito (5 execuções)')
plt.xlabel('f1 (Custo de manutenção)')
plt.ylabel('f2 (Impacto de falha)')
plt.grid(True)
plt.legend()
plt.show()
